# HER2 One Scanner, 5-Fold (80/20) split, Categorical from Mean, Epoch at automatic Stop, Dropout (0.7)

In [97]:
import numpy as np
import pandas as pd
import random
from keras.callbacks import EarlyStopping
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report
import csv
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt

In [98]:
#For single scanner
BASE_PATH = '/home/diam/Desktop/HER2_data_categorical/Aperio_FDA/'
#BASE PATH for working from home:
#BASE_PATH = '/home/OSEL/Desktop/HER2_data_categorical/'
#epochs = 10
batch_size = 32
num_classes = 3
#epochs = 35

## Get Data - Practice

In [99]:
#This is the version from Ravi's code:

#FDA
#X_FDA = []
#idx_FDA = []
#for index, image_filename in list(enumerate(BASE_PATH)):
#	img_file = cv2.imread(BASE_PATH + '/' + image_filename)
#	if img_file is not None:
		#img_file = smisc.imresize(arr = img_file, size = (600,760,3))
#		img_file = smisc.imresize(arr = img_file, size = (120,160,3))		
#		img_arr = np.asarray(img_file)
#		X_FDA.append(img_arr)
#		idx_FDA.append(index)

#X_FDA = np.asarray(X_FDA)
#idx_FDA = np.asarray(idx_FDA)
#random.seed(rs)
#random_id = random.sample(idx_FDA, len(idx_FDA)/2)
#random_FDA = []
#for i in random_id:
#	random_FDA.append(X_FDA[i])

#random_FDA = np.asarray(random_FDA)

## Get Data - Real

In [100]:
def get_data(folder):
    X = []
    y = []
    filenames = []

    for hclass in os.listdir(folder):
        if not hclass.startswith('.'):
            if hclass in ["0"]:
                label = 0
            else: #label must be 1 or 2
                if hclass in ["1"]:
                    label = 1
                else:
                    label = 2
            for image_filename in os.listdir(folder + hclass):
                filename = folder + hclass + '/' + image_filename
                img_file = cv2.imread(folder + hclass + '/' + image_filename)
                
                if img_file is not None:
                    img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
                    filenames.append(filename)

    X = np.asarray(X)
    y = np.asarray(y)
    z = np.asarray(filenames)
    return X,y,filenames

In [101]:
X, y, z = get_data(BASE_PATH)


#print(X)
#print(y)
#print(z)
print(len(X))
print(len(y))
print(len(z))

#INTEGER ENCODE
#https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
encoder = LabelEncoder()
y_cat = np_utils.to_categorical(encoder.fit_transform(y))
print(y_cat)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


241
241
241
[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.

### Old Code

In [102]:

#encoder = LabelEncoder()
#encoder.fit(y)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

#encoded_y_train = encoder.transform(y_train)
#encoded_y_test = encoder.transform(y_test)

#y_train = np_utils.to_categorical(encoded_y_train)
#y_test = np_utils.to_categorical(encoded_y_test)

In [103]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Fit Model with K-Fold X-Val

In [104]:
kf = KFold(n_splits = 5, random_state=5, shuffle=True)
print(kf.get_n_splits(y))
print(kf)


#for train_index, test_index in kf.split(y):
#    X_train, X_test = X[train_index], X[test_index]
#    print(train_index, test_index)




5
KFold(n_splits=5, random_state=5, shuffle=True)


In [105]:

oos_y = []
oos_pred = []
fold = 0


for train, test in kf.split(y_cat):
    fold+=1
    print("fold #{}".format(fold))
        
    X_train = X[train]
    y_train = y_cat[train]
    X_test = X[test]
    y_test = y_cat[test]
    
    #encoder = LabelEncoder()
    #encoder.fit(y_test)
    #y_train = np_utils.to_categorical(encoder.transform(y_train))
    #y_test = np_utils.to_categorical(encoder.transform(y_test))
    
    model = Sequential()
    model.add(Lambda(lambda x: x * 1./255., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
    
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')
    
    model.fit(
            X,
            y_cat,
            validation_data=(X_test,y_test),
            callbacks=[monitor],
            shuffle=True,
            batch_size=batch_size,
            verbose=1,
            epochs=1000)
        
    pred = model.predict(X_test)
        
    oos_y.append(y_test)
    pred = np.argmax(pred,axis=1)
    oos_pred.append(pred)
    
        #measure the fold's accuracy
    y_compare = np.argmax(y_test,axis=1) #for accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print("Fold Score (accuracy): {}".format(score))
    print(pred)

fold #1
Train on 241 samples, validate on 49 samples
Epoch 1/1000
241/241 [==============================] - 1s 5ms/step - loss: 1.3395 - acc: 0.4855 - val_loss: 1.0304 - val_acc: 0.4694
Epoch 2/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.9503 - acc: 0.5270 - val_loss: 1.0840 - val_acc: 0.4082
Epoch 3/1000
241/241 [==============================] - 0s 1ms/step - loss: 1.0076 - acc: 0.5270 - val_loss: 0.9155 - val_acc: 0.7959
Epoch 4/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.9535 - acc: 0.5477 - val_loss: 0.9935 - val_acc: 0.4694
Epoch 5/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9313 - acc: 0.6390 - val_loss: 0.7477 - val_acc: 0.7755
Epoch 6/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8638 - acc: 0.6307 - val_loss: 0.7301 - val_acc: 0.8163
Epoch 7/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8455 - acc: 0.7261 - val_loss: 0.7109 - val_acc: 0.7143
Epoch 8/1

241/241 [==============================] - 0s 2ms/step - loss: 0.3459 - acc: 0.8755 - val_loss: 1.0232 - val_acc: 0.6327
Epoch 62/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3607 - acc: 0.8880 - val_loss: 0.2025 - val_acc: 0.8776
Epoch 63/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1895 - acc: 0.9129 - val_loss: 0.1488 - val_acc: 0.8776
Epoch 64/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1725 - acc: 0.9004 - val_loss: 0.4942 - val_acc: 0.8776
Epoch 65/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6970 - acc: 0.8091 - val_loss: 0.3335 - val_acc: 0.8367
Epoch 66/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2603 - acc: 0.8880 - val_loss: 0.1676 - val_acc: 0.8776
Epoch 67/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1698 - acc: 0.9087 - val_loss: 0.1545 - val_acc: 0.8776
Epoch 68/1000
241/241 [==============================] - 0s 1ms/step 

241/241 [==============================] - 0s 1ms/step - loss: 0.0586 - acc: 0.9834 - val_loss: 0.0082 - val_acc: 1.0000
Epoch 122/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0526 - acc: 0.9793 - val_loss: 1.1575 - val_acc: 0.7959
Epoch 123/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7414 - acc: 0.8838 - val_loss: 0.0291 - val_acc: 1.0000
Epoch 124/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0796 - acc: 0.9710 - val_loss: 0.0195 - val_acc: 1.0000
Epoch 125/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0807 - acc: 0.9793 - val_loss: 0.0325 - val_acc: 1.0000
Epoch 126/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0633 - acc: 0.9834 - val_loss: 0.0208 - val_acc: 1.0000
Epoch 127/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0520 - acc: 0.9793 - val_loss: 0.0094 - val_acc: 1.0000
Epoch 128/1000
241/241 [==============================] - 0s 1m

241/241 [==============================] - 0s 1ms/step - loss: 0.5460 - acc: 0.8340 - val_loss: 0.6504 - val_acc: 0.7500
Epoch 21/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6404 - acc: 0.7884 - val_loss: 0.4585 - val_acc: 0.8542
Epoch 22/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4881 - acc: 0.8506 - val_loss: 1.2275 - val_acc: 0.5625
Epoch 23/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8436 - acc: 0.7178 - val_loss: 0.4760 - val_acc: 0.8333
Epoch 24/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4754 - acc: 0.8506 - val_loss: 0.4701 - val_acc: 0.8333
Epoch 25/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.4284 - acc: 0.8672 - val_loss: 0.4823 - val_acc: 0.7917
Epoch 26/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.4449 - acc: 0.8589 - val_loss: 0.4170 - val_acc: 0.8333
Epoch 27/1000
241/241 [==============================] - 0s 2ms/step 

Epoch 81/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5143 - acc: 0.8755 - val_loss: 0.2253 - val_acc: 0.8750
Epoch 82/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1856 - acc: 0.9170 - val_loss: 0.1764 - val_acc: 0.8750
Epoch 83/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1703 - acc: 0.9046 - val_loss: 0.1731 - val_acc: 0.8750
Epoch 84/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1710 - acc: 0.9004 - val_loss: 0.1310 - val_acc: 0.8958
Epoch 85/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1557 - acc: 0.8963 - val_loss: 0.1811 - val_acc: 0.9792
Epoch 86/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1551 - acc: 0.9129 - val_loss: 0.1273 - val_acc: 0.9583
Epoch 87/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.3793 - acc: 0.8963 - val_loss: 0.1639 - val_acc: 0.8750
Epoch 88/1000
241/241 [==============================] 

...

Epoch 103/1000
241/241 [==============================] - 0s 2ms/step - loss: 1.7866 - acc: 0.8133 - val_loss: 0.0935 - val_acc: 0.9792
Epoch 104/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1227 - acc: 0.9710 - val_loss: 0.0441 - val_acc: 1.0000
Epoch 105/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0826 - acc: 0.9710 - val_loss: 0.0570 - val_acc: 0.9792
Epoch 106/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1192 - acc: 0.9336 - val_loss: 0.0322 - val_acc: 1.0000
Epoch 107/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0615 - acc: 0.9917 - val_loss: 0.0223 - val_acc: 1.0000
Epoch 108/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6026 - acc: 0.8797 - val_loss: 0.0527 - val_acc: 0.9792
Epoch 109/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0949 - acc: 0.9585 - val_loss: 0.0292 - val_acc: 1.0000
Epoch 110/1000
241/241 [========================

241/241 [==============================] - 0s 2ms/step - loss: 0.0350 - acc: 0.9834 - val_loss: 0.0122 - val_acc: 1.0000
Epoch 164/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5341 - acc: 0.9253 - val_loss: 0.0197 - val_acc: 1.0000
Epoch 165/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0761 - acc: 0.9751 - val_loss: 0.0101 - val_acc: 1.0000
Epoch 166/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0480 - acc: 0.9834 - val_loss: 0.0079 - val_acc: 1.0000
Epoch 167/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0505 - acc: 0.9876 - val_loss: 0.0112 - val_acc: 1.0000
Epoch 168/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0163 - acc: 1.0000 - val_loss: 0.0034 - val_acc: 1.0000
Epoch 169/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0289 - acc: 0.9917 - val_loss: 0.0014 - val_acc: 1.0000
Epoch 170/1000
241/241 [==============================] - 0s 1m

241/241 [==============================] - 0s 2ms/step - loss: 0.5126 - acc: 0.8050 - val_loss: 0.6114 - val_acc: 0.8125
Epoch 29/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.4644 - acc: 0.8423 - val_loss: 0.2810 - val_acc: 0.8958
Epoch 30/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.4222 - acc: 0.8548 - val_loss: 0.3908 - val_acc: 0.8333
Epoch 31/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.3692 - acc: 0.8506 - val_loss: 0.2260 - val_acc: 0.8958
Epoch 32/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4916 - acc: 0.8299 - val_loss: 0.3193 - val_acc: 0.8750
Epoch 33/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.4653 - acc: 0.8299 - val_loss: 0.2488 - val_acc: 0.9167
Epoch 34/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4121 - acc: 0.8423 - val_loss: 0.4951 - val_acc: 0.8542
Epoch 35/1000
241/241 [==============================] - 0s 2ms/step 

Epoch 89/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1016 - acc: 0.9419 - val_loss: 0.0251 - val_acc: 1.0000
Epoch 90/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1308 - acc: 0.9336 - val_loss: 0.0403 - val_acc: 0.9792
Epoch 91/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.4928 - acc: 0.8672 - val_loss: 0.1607 - val_acc: 0.9375
Epoch 92/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2305 - acc: 0.9129 - val_loss: 0.0565 - val_acc: 0.9792
Epoch 93/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1194 - acc: 0.9419 - val_loss: 0.0322 - val_acc: 1.0000
Epoch 94/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0691 - acc: 0.9751 - val_loss: 0.0206 - val_acc: 1.0000
Epoch 95/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5026 - acc: 0.8340 - val_loss: 0.0680 - val_acc: 0.9583
Epoch 96/1000
241/241 [==============================] 

241/241 [==============================] - 0s 2ms/step - loss: 0.0281 - acc: 0.9959 - val_loss: 5.5893e-04 - val_acc: 1.0000
Epoch 150/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1896 - acc: 0.9502 - val_loss: 0.0209 - val_acc: 1.0000
Epoch 151/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0643 - acc: 0.9793 - val_loss: 0.0061 - val_acc: 1.0000
Epoch 152/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0671 - acc: 0.9793 - val_loss: 0.0053 - val_acc: 1.0000
Epoch 153/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0383 - acc: 0.9834 - val_loss: 3.9784e-04 - val_acc: 1.0000
Epoch 154/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0371 - acc: 0.9917 - val_loss: 0.0108 - val_acc: 1.0000
Epoch 155/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.6820 - acc: 0.8797 - val_loss: 0.0553 - val_acc: 0.9792
Epoch 156/1000
241/241 [==============================]